In [2]:
import os
import json
import shlex
import subprocess
import uuid
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from openai import OpenAI as OpenAIClient

current_story_section = None
current_story_section_index = 0

def read_prompt_from_md(filename):
    """Read the content of a markdown file."""
    with open(filename, 'r') as file:
        return file.read().strip()

def temporary_save(content):
    """Save content to a temporary file."""
    temp_name = f"temp_{uuid.uuid4()}.txt"
    with open(temp_name, 'w') as file:
        file.write(content)

def trim_json_string(json_string):
    """Trim markdown formatting from a JSON string."""
    json_string = json_string.replace("```json", "").replace("```", "").strip()
    return json_string

def validate_json(pattern, json_string):
    """Validate and fix JSON string if necessary."""

    import glob 
    pattern_count = len(glob.glob(f"{pattern}_*.json"))
    pattern_count += 1
    json_string = trim_json_string(json_string)

    try:
        json_data = json.loads(json_string)
        with open(f"{pattern}_{pattern_count}.json", 'w') as file:
            json.dump(json_data, file, indent=2)
        return json_data
    except json.JSONDecodeError:
        print("Invalid JSON. Attempting to fix...")

        escaped_prompt = shlex.quote(json_string)
        cmd = ["/Users/jkatigbak/go/bin/fabric", "--pattern", "validate_json"]
        process = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)   
        stdout, stderr = process.communicate(input=json_string)
        result = subprocess.CompletedProcess(cmd, process.returncode, stdout, stderr)
        print(result.stdout)
        
        try: 
            json_string = trim_json_string(result.stdout)
            json_data = json.loads(json_string)
            with open(f"fixed_{pattern}_{pattern_count}.json", 'w') as file:
                json.dump(json_data, file, indent=2)
            return json_data
        except json.JSONDecodeError:
            print("Failed to fix JSON.")
            return None
        
def write_artifact(pattern, prompt, content):
    """Write the generated content to a file with an incremented index."""
    import glob
    files = glob.glob(f"{pattern}_*.txt")
    file_idx = len(files) + 1
    with open(f"{pattern}_{file_idx}.txt", "w") as file:
        file.write(prompt)
        file.write("\n\n===================\n\n")
        file.write(content)

def read_template(filename):
    """Read a JSON template from a file."""
    with open(filename, 'r') as file:
        return json.load(file)

def chat_completion(pattern, user_prompt):
    """Get a response from the chat model based on the user prompt."""
    try:
        # Escape special characters in user_prompt
        escaped_prompt = shlex.quote(user_prompt)
        
        # Use a list of arguments instead of shell=True for better security
        cmd = ["/Users/jkatigbak/go/bin/fabric", "--pattern", pattern]
        
        # Use subprocess.Popen to pipe the input
        process = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        
        # Send the user_prompt to stdin and get the output
        stdout, stderr = process.communicate(input=escaped_prompt)
        
        result = subprocess.CompletedProcess(cmd, process.returncode, stdout, stderr)
        write_artifact(pattern, user_prompt, result.stdout)
        return result.stdout
    except Exception as e:
        print(f"Error in chat completion: {e}")
        return None

def generate_story_outline(story_parameters):
    """Generate a story outline based on provided parameters."""
    user_prompt = f"These are the story parameters to fulfill your system prompt: {json.dumps(story_parameters)}, please output the outline requested."
    outline = chat_completion("story_outline", user_prompt)
    print(outline)
    return outline

def enhance_story_outline(initial_outline):
    user_prompt = f"Enhance this story outline by adding more details to each section into a valid JSON object: \n\n====\n\n{initial_outline}\n\n====\n\n Please ensure that the output is valid JSON."
    enhanced = chat_completion("story_outline_enhancement", user_prompt)
    print(enhanced)
    return validate_json("story_outline_enhancement", enhanced)

def develop_story_section(section_outline, story_outline):
    """Develop a specific section of the story based on the outline."""
    user_prompt = f"""This was the story outline that was used to create this section: \n\n{story_outline}\n\nSection Outline for you to develop: \n\n{json.dumps(section_outline)}\n\nPlease develop the section with the section provided, ensuring that the story remains cohesive and the plot points are well integrated."""   
    developed = chat_completion("story_development", user_prompt)
    print(developed)
    return developed

def pair_image_and_text(story_section):
    """Pair images with the corresponding text in the story section."""
    user_prompt = f"This is the story section to pair the image with: {json.dumps(story_section)}"
    paired = chat_completion("image_text_pairing", user_prompt)
    print(paired)
    return validate_json("image_text_pairing", paired)

def create_image_generation_prompt(image_description):
    """Create a prompt for image generation based on the description."""
    user_prompt = f"This is the image description object to create an image prompt for: {json.dumps(image_description)}, the output will be simple text without any formatting."
    prompt = chat_completion("image_generation", user_prompt)
    print(f"Image Prompt: {prompt}")
    return prompt

def get_api_key():
    """Retrieve the OpenAI API key from environment or file."""
    if api_key := os.environ.get('OPENAI_API_KEY'):
        return api_key
    with open('/Users/jkatigbak/.config/openai', 'r') as file:
        return file.read().strip()

def generate_image(prompt):
    """Generate an image based on the provided prompt."""
    try:
        api_key = get_api_key()
        print(api_key)
        os.environ['OPENAI_API_KEY'] = api_key
        # Generate the image
        client = OpenAIClient(api_key=api_key)
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            quality="standard",
            n=1,
        )

        return response.data[0].url

    except Exception as e:
        print(f"Error generating image: {e}")
        return None

def output_results(full_story, storybook_content, image_prompts, image_urls):
    """Output the results to a JSON file."""
    with open('storybook_output.json', 'w') as f:
        json.dump({
            'full_story': full_story,
            'storybook_content': storybook_content,
            'image_prompts': image_prompts,
            'image_urls': image_urls
        }, f, indent=2)
    print("Results saved to storybook_output.json")



In [3]:
story_parameters = read_template('story_template.json')
initial_outline = generate_story_outline(story_parameters)


```json
{
  "ageRange": "5-7 years",
  "storyLength": "1000 words",
  "themes": ["Imagination", "Creativity", "Friendship"],
  "moralOfTheStory": "Using your imagination can turn the ordinary into something extraordinary.",
  "characters": [
    {
      "name": "Luna",
      "description": "A curious little girl with a vivid imagination"
    },
    {
      "name": "Bubbles",
      "description": "A friendly blue dragon who loves to fly"
    },
    {
      "name": "Oliver",
      "description": "A shy but clever boy who enjoys building things"
    }
  ],
  "storySetting": "A whimsical village surrounded by enchanting hills and sparkling rivers",
  "tone": "Playful and inspiring",
  "parentalConcerns": [
    "Ensure all characters are positive role models",
    "Avoid any frightening elements"
  ],
  "inclusionAndDiversity": "Include characters from different backgrounds and cultures, showcasing diverse talents",
  "ending": "Uplifting ending that inspires creativity and friendship",
  "

In [4]:
enhanced_outline = enhance_story_outline(initial_outline)

```json
{
  "enhancedOutline": {
    "Introduction": {
      "plotPoints": [
        "Luna is seen daydreaming under a big colorful tree, imagining herself as a brave explorer.",
        "Bubbles, the friendly blue dragon, swoops down from the sky, playfully blowing bubbles that float around Luna.",
        "Oliver is in his workshop, surrounded by quirky inventions like a toy robot and a kite made of colorful paper, feeling shy as he watches Luna and Bubbles from a distance.",
        "The whimsical village is alive with colorful houses adorned with flowers, with children playing and laughing, showcasing the vibrant community."
      ],
      "characterDevelopment": {
        "Luna": "Portrayed as adventurous and imaginative, always dreaming of new possibilities.",
        "Bubbles": "Demonstrates a playful spirit and a love for friendship, eager to join Luna's adventures.",
        "Oliver": "Introduced as shy but clever, with a passion for building, hinting at his potential to grow 

In [6]:
def process_story_section(section_name, enhanced_outline, initial_outline):
    story_section = develop_story_section({section_name: enhanced_outline['enhancedOutline'][section_name]}, initial_outline)
    
    paired_image_and_text = pair_image_and_text(story_section)
    
    image_prompt = create_image_generation_prompt(paired_image_and_text['imageDescription'])
    
    image = generate_image(image_prompt)
    
    return {
        'story_section': story_section,
        'pair_image_and_text': paired_image_and_text,
        'image_prompt': image_prompt,
        'image': image
    }


# Process Introduction section
introduction_result = process_story_section('Introduction', enhanced_outline, initial_outline)

# introduction_story_section = develop_story_section({'Introduction': enhanced_outline['enhancedOutline']['Introduction']}, initial_outline)
# paired_image_and_text = pair_image_and_text(introduction_story_section)


### Introduction

In a whimsical village filled with colorful houses and vibrant flowers, there was a big, colorful tree that stood proudly in the center of a sunlit meadow. Underneath its wide branches, a curious little girl named Luna sat daydreaming. With her bright eyes sparkling with excitement, she imagined herself as a brave explorer, sailing across oceans of candy and climbing mountains made of marshmallows. 

As she closed her eyes, her imagination painted vivid pictures of daring adventures. "I could discover magical lands and meet friendly creatures!" she whispered to herself, a smile dancing on her lips. Just then, a swoosh of air rustled the leaves above her, and a friendly blue dragon named Bubbles swooped down from the sky. Bubbles was not just any dragon; he loved to play, and today was no different. 

“Look out below!” he called out with a cheerful roar, blowing bubbles that floated gracefully around Luna like tiny rainbows. Each bubble shimmered in the sunlight, refle

In [7]:
introduction_result 

{'story_section': '### Introduction\n\nIn a whimsical village filled with colorful houses and vibrant flowers, there was a big, colorful tree that stood proudly in the center of a sunlit meadow. Underneath its wide branches, a curious little girl named Luna sat daydreaming. With her bright eyes sparkling with excitement, she imagined herself as a brave explorer, sailing across oceans of candy and climbing mountains made of marshmallows. \n\nAs she closed her eyes, her imagination painted vivid pictures of daring adventures. "I could discover magical lands and meet friendly creatures!" she whispered to herself, a smile dancing on her lips. Just then, a swoosh of air rustled the leaves above her, and a friendly blue dragon named Bubbles swooped down from the sky. Bubbles was not just any dragon; he loved to play, and today was no different. \n\n“Look out below!” he called out with a cheerful roar, blowing bubbles that floated gracefully around Luna like tiny rainbows. Each bubble shimmer

'In the heart of a vibrant meadow, where daisies danced in the gentle breeze and sunflowers stood tall like cheerful sentinels, there lived a shy little rabbit named Luna. She loved to daydream, her imagination a tapestry of adventures and friendships. But on this sunny day, as she peeked out from behind a blooming flower, her heart fluttered with uncertainty. \n\nLuna watched her friends play tag, their laughter ringing like sweet chimes in the air. Benny, the energetic bear, rolled down a grassy hill, his joyful giggles echoing as he tumbled to a stop, his fur glistening in the sun. “Whee! Come on, everyone!” he called out, inviting all the animals to join in the fun. Nearby, Tilly, the wise turtle, shared stories of brave explorers with a group of younger animals, her voice warm and soothing. “And they crossed rivers and climbed mountains, all with courage in their hearts!” she narrated, her eyes sparkling like the river that flowed gently nearby.\n\nAs Luna watched, her heart ached

In [233]:
problem_result = process_story_section('Conflict', enhanced_outline, initial_outline)


### Conflict/Problem

On a sunny day in the vibrant meadow, Benny the cheerful bear stood atop a large, sun-warmed rock, his eyes sparkling with excitement. "Friends, gather around!" he called, waving his paws in the air. Luna, the shy little rabbit, sat under a blossoming cherry tree, her heart fluttering like the butterflies dancing nearby. She watched her friends—Benny and Tilly—wondering what grand adventure Benny was planning this time.

Benny's voice rang out again, "I have the most exciting idea! Let’s explore the magical Whispering Woods! It’s a land filled with wonders, sparkles, and sounds that twinkle like stars!" His eyes shone with anticipation as he painted a picture of the enchanted forest.

As Benny spoke, Luna’s heart raced. She imagined the tall trees whispering secrets, the shimmering lights of fireflies, and the unknown creatures lurking behind every bush. "What if I get lost?" she thought, her little ears drooping slightly. "What if I hear strange sounds and can't 

In [234]:
rising_action_story_section = develop_story_section({'RisingAction': enhanced_outline['enhancedOutline']['RisingAction']}, initial_outline)

As Luna, Benny, and Tilly stepped into the Whispering Woods, the world around them transformed in the most magical way. The trees, tall and ancient, seemed to lean in closer, their leaves rustling softly like they were sharing secrets just for the trio. Sunlight danced through the branches, creating playful patterns on the forest floor, and a gentle breeze whispered sweet melodies that made the air feel alive. Luna took a deep breath, her heart fluttering with a mix of excitement and nervousness.

“Look at the colors!” Benny exclaimed, his eyes sparkling with wonder as he pointed at the vibrant flowers peeking through the underbrush. “This place is amazing!”

But just as they ventured deeper, they came upon their first challenge: a bubbling stream with slippery stepping stones scattered across the water. The stones glistened like little jewels, but Luna’s tummy twisted into knots at the thought of crossing. 

“I can’t do it,” she murmured, taking a small step back. 

Benny turned to he

In [235]:
climax_story_section = develop_story_section({'Climax': enhanced_outline['enhancedOutline']['Climax']}, initial_outline)

### Climax

The sun shone brightly as Luna, Benny, and Tilly stepped into a beautiful clearing filled with wildflowers dancing in the gentle breeze. The vibrant colors of the flowers—reds, yellows, and purples—painted the meadow like a joyful rainbow. In the center of this magical place, something sparkled under the sun. It was a treasure chest, glimmering with golden light, and Luna's heart raced with excitement.

But as they approached, they noticed a little fox peeking out from behind a bush. Its bright orange fur shimmered in the sunlight, but its wide eyes were filled with uncertainty. The fox trembled, unsure of what to make of the visitors. “Who are they? What do they want?” it seemed to think, retreating further into the shadows.

Luna’s heart sank. She remembered how scared she felt when Benny first invited her on this adventure. “What if the fox is just like me?” she whispered to herself, feeling a mix of fear and empathy. She glanced at Benny, who was already taking a step t

In [236]:
falling_action_story_section = develop_story_section({'FallingAction': enhanced_outline['enhancedOutline']['FallingAction']}, initial_outline)


KeyboardInterrupt: 

In [ ]:
resolution_story_section = develop_story_section({'Resolution': enhanced_outline['enhancedOutline']['Resolution']}, initial_outline)

In [ ]:
full_story = story_development(enhanced_outline)

/var/folders/8z/rx0dw8cj6d35_d3s4m5ffwlc0000gn/T/ipykernel_55716/3844839285.py:1: RuntimeWarning: coroutine 'story_development' was never awaited
  full_story = story_development(enhanced_outline)


In [ ]:
def main():
    # Read input from template file
    story_parameters = read_template('story_template.json')

    # Generate initial story outline
    initial_outline = generate_story_outline(story_parameters)
    if not initial_outline:
        print("Failed to generate valid story outline.")
        return

    # Enhance story outline
    enhanced_outline = enhance_story_outline(initial_outline)
    if not enhanced_outline:
        print("Failed to enhance story outline.")
        return

    # Develop full story
    full_story = []
    for section in enhanced_outline['sections']:
        developed_section = develop_story_section(section)
        if developed_section:
            full_story.append(developed_section)
        else:
            print(f"Failed to develop section: {section}")

    # Pair images and text
    storybook_content = []
    for section in full_story:
        paired_content = pair_image_and_text(section)
        if paired_content:
            storybook_content.append(paired_content)
        else:
            print(f"Failed to pair image and text for section: {section}")

    # Create image generation prompts
    image_prompts = []
    for content in storybook_content:
        prompt = create_image_generation_prompt(content['image_description'])
        if prompt:
            image_prompts.append(prompt)
        else:
            print(f"Failed to create image generation prompt for: {content['image_description']}")

    # Generate images
    image_urls = []
    for prompt in image_prompts:
        image_url = generate_image(prompt['prompt'])
        if image_url:
            image_urls.append(image_url)
        else:
            print(f"Failed to generate image for prompt: {prompt}")

    # Output the results
    output_results(full_story, storybook_content, image_prompts, image_urls)


In [ ]:
# check which fabric is being used
print(subprocess.run("fabric --version", shell=True, capture_output=True, text=True))


CompletedProcess(args='fabric --version', returncode=127, stdout='', stderr='/bin/sh: fabric: command not found\n')
